# Flomaster Example: Single Pump

In [1]:
from functools import partial 

from numpy import trapz
import plotly.graph_objects as go
from scipy.optimize import Bounds, differential_evolution

from fmu_optimise import create_cost_function

In [2]:
filename = "SinglePump.fmu"

# set input parameters with bounds
inputs = {
    "[TopDiameter]": [0.01, 0.3],
    "[BottomDiameter]": [0.01, 0.3],
    "[BottomHeatIn]": [80, 150],
    "[TopHeatIn]": [80, 150],
}

# outputs to be recorded
output = ["Pump Power"]

# create bounds in expected format
bounds = Bounds([b[0] for b in inputs.values()], [b[1] for b in inputs.values()])

# create post simulation cost function
def post_simulation_function(results):
    return trapz(results["Pump Power"])

In [3]:
# cost function
func = create_cost_function(
    filename=filename,
    input_names=inputs.keys(),
    output_names=output,
    start_time=0,
    stop_time=60,
    step_size=0.5,
    post_simulation_function=post_simulation_function
)

In [4]:
def unbound_generation_callback(fig, convergence_list, xk, convergence=0):
    print(xk)
    convergence_list.append(convergence)
    fig.data[0].y = convergence_list

fig = go.FigureWidget()
fig.add_scatter()
fig
convergence_list = []
generation_callback = partial(unbound_generation_callback, fig, convergence_list)
# solve
result = differential_evolution(func, bounds, maxiter=20, callback=generation_callback)
print(result)

[OK] Starting server /tmp/tmpt5fn629d/resources/linux64/Flomaster.Fmi.EmbeddedSolver
[OK] Started with socket unix:/tmp/FMI_HOST_bb0ec8d2cfbd4a2baa13bf73fc571c6f.tmp
[OK] About to initialise FmiClient in client...
[OK] Called InstantiateInstance in fmi communication client
[OK] Called SetupExperiment in fmi communication client
[OK] Called SetReal in fmi communication client
[OK] Called SetReal in fmi communication client
[OK] Called SetReal in fmi communication client
[OK] Called SetReal in fmi communication client
[OK] Called EnterInitializationMode in fmi communication client
[ERROR] Licensing Failed to Initialize: MGLS runtime error license_request_failed (515)
[OK] Called ExitInitializationMode in fmi communication client
[OK] Called GetReal in fmi communication client
[ERROR] License check has requested cancellation of the simulator.
Please check SALT_LICENSE_SERVER environment variable.
[ERROR] Called DoStep in fmi communication client


Exception: fmi2DoStep failed with status 3.